<a href="https://colab.research.google.com/github/Amarantine-xiv/Another-FF14-Combat-Sim/blob/main/CoreSimulator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Copyright 2023 A. Falena

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

In [10]:
VERSION=0.04

In [11]:
#@title Imports

import numpy as np
import matplotlib.pyplot as plt
import time
import copy
from dataclasses import dataclass
import heapq
from enum import Enum

In [12]:
#@title Game Constants
@dataclass(frozen=True)
class GameConsts:
  gcd_recast_time: float = 2500
  damage_snapshot_time_before_cast_finishes = 500

In [13]:
#@title CastSpec

@dataclass(frozen=True)
class CastSpec:
  cast_time: int
  is_GCD: bool
  gcd_base_recast_time: int = None #after you use this skill, how long until you can use a gcd?
  base_recast_time: int = 0 # the base "cooldown" of the skill- when can you use it next?
  animation_lock: int = 65 #does not take into account ping

  def __post_init__(self):
    if self.gcd_base_recast_time is None:
      if self.is_GCD:
        # assume a default of 2500ms recast time for all gcds, unless otherwise stated
        object.__setattr__(self, 'gcd_base_recast_time', GameConsts.gcd_recast_time)
      else:
        # By default, non-gcds don't affect gcd_base_recast_time. If this is not
        # true for this skill, then set gcd_base_recast_time explicitly.
        object.__setattr__(self, 'gcd_base_recast_time', 0)

  def __str__(self):
    res = "Cast time: {}".format(self.cast_time)
    return res

In [14]:
#@title RotationBuilderUtil

class RotationBuilderUtil:
  """ A utility class used to turn 1) a series of button (skills) pressed and optionally the specific time they were pressed, and 2) proc application times into a series of damage instances and applicable buffs/debuffs."""
  def __init__(self, stats):
    self._stats = stats
    self._q = []
    self._q_timed = []

  def add_skill(self, skill_name, t=None):
    """Adds a skill to be used. All skills are considered to be used sequential at the first available opportunity, unless a time t is specified."""
    if t is None:
      heapq.heappush(self._q, skill_name)
    else:
      heapq.heappush(self._q_timed, skill_name)

In [15]:
#@title Skill, SkillLibrary

class Skill:
  def __init__(self, name):
    self.name = name
    self.cast_spec = None

  def __str__(self):
    return self.name

class SkillLibrary:
  def __init__(self):
    self._skills = {}

  def add_job_class(self, job_name):
    self._skills[job_name] = {}

  def add_skill(self, job_name, skill):
    skill_name = skill.name
    if skill_name in self._skills[job_name]:
      raise RuntimeError('Duplicate skill being added to the skill library (this is probably a naming error). Job: {}, Skill name: {}'.format(job_name, skill_name))
    self._skills[job_name][skill.name] = skill

  def print_skills(self):
    for job_name in self._skills:
      for skill_name in self._skills[job_name]:
        print("Job name: {}, Skill name: {}".format(job_name, skill_name))

In [16]:
#@title Test utils

class TestClass:
  def __init__(self):
    self.test_fns = []

  def is_a_test(f):
    f.__is_a_test__ = True
    return f

  def _get_test_methods(self):
    test_methods = []
    for fn_name in dir(self):
      fn = getattr(self, fn_name)
      if getattr(fn, '__is_a_test__', False):
        test_methods.append(fn)
    return test_methods

  def _print_result(self, passing, failing):
    print('Testing for {}. {}/{} tests passed.'.format(self.__class__.__name__, len(passing), len(passing)+len(failing)))
    if len(failing) > 0:
      print('Failing tests:')
      for test_info in failing:
        print("{}: {}\n".format(test_info[0], test_info[1]))
    print('Passing tests:')
    for test_name in passing:
      print("{}".format(test_name))

  def run_single(self, test_name):
    for test_fn in self._get_test_methods():
      if test_name == test_fn.__name__:
        test_passed, err_msg = test_fn()
        if test_passed:
          print('{} passed!'.format(test_name))
          return
        else:
          print('{} failed: {}'.format(test_name, err_msg))
          return
    print('No test found with name {}'.format(test_name))

  def run_all(self):
    passing = []
    failing = []
    for test_fn in self._get_test_methods():
      test_name = test_fn.__name__
      test_passed, err_msg = test_fn()
      if (test_passed):
        passing.append(test_name)
      else:
        failing.append((test_name, err_msg))
    self._print_result(passing, failing)


In [17]:
class CastSpecTest(TestClass):
  @TestClass.is_a_test
  def gcd_override_gcd_test(self):
    err_msg=""
    test_passed = True
    cast_spec = CastSpec(cast_time=2000, is_GCD=True)
    if cast_spec.gcd_base_recast_time != 2500:
      err_msg = "Recast time was expected to be 2500, but it was {}".format(cast_spec.gcd_base_recast_time)
      test_passed = False
    return test_passed, err_msg

  @TestClass.is_a_test
  def gcd_override_ogcd_test(self):
    err_msg=""
    test_passed = True
    cast_spec = CastSpec(cast_time=2000, is_GCD=False)
    if cast_spec.gcd_base_recast_time != 0:
      err_msg = "Recast time was expected to be 0, but it was {}".format(cast_spec.gcd_base_recast_time)
      test_passed = False
    return test_passed, err_msg

  @TestClass.is_a_test
  def gcd_override_set_recast_time(self):
    err_msg=""
    test_passed = True
    cast_spec = CastSpec(cast_time=2000, gcd_base_recast_time=1500, base_recast_time=60000, is_GCD=False)
    if cast_spec.gcd_base_recast_time != 1500:
      err_msg = "Recast time was expected to be 1500, but it was {}".format(cast_spec.gcd_base_recast_time)
      test_passed = False
    return test_passed, err_msg


In [18]:
CastSpecTest().run_all()

Testing for CastSpecTest. 3/3 tests passed.
Passing tests:
gcd_override_gcd_test
gcd_override_ogcd_test
gcd_override_set_recast_time
